In [1]:
!pip install tf-nightly-gpu
try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass
tf.enable_v2_behavior()
print(tf.__version__)
import tensorflow as tf
tf.config.list_physical_devices("GPU")

     |████████████████████████████████| 516.0MB 33kB/s 
     |████████████████████████████████| 2.8MB 58.9MB/s 
     |████████████████████████████████| 460kB 12.7MB/s 
     |████████████████████████████████| 2.9MB 53.3MB/s 
     |████████████████████████████████| 778kB 46.9MB/s 
ERROR: tensorflow 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
  Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Found existing installation: h5py 2.8.0
    Uninstalling h5py-2.8.0:
      Successfully uninstalled h5py-2.8.0
2.2.0-dev20200303


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import pandas as pd


df = pd.read_csv('https://raw.githubusercontent.com/Build-Week-AirBnB-Optimal-Price-2/Data-Science/master/Notebook%20Folders/data/airbnb%20data%20cleaned.csv')
df.head()

,Unnamed: 0,accommodates,bathrooms,bedrooms,price,cleaning_fee,security_deposit,extra_people,guests_included,distance,size,host_is_superhost,latitude,longitude,minimum_nights,instant_bookable,is_business_travel_ready,tv,wifi,kitchen,washer,dryer,heating,free_parking,smoking_allowed,property_type,neighbourhood,room_type,cancellation_policy,bed_type
0,0,3,1.0,1,60.0,30.0,200.0,28.0,1,1.624906,75.0,True,52.534537,13.402557,4,False,False,True,True,True,True,True,True,False,False,0,1,1,0,0
1,1,2,1.0,1,17.0,0.0,0.0,0.0,1,3.170627,25.0,False,52.548513,13.404553,2,False,False,False,True,True,True,True,True,False,False,1,2,0,1,0
2,2,4,1.0,1,90.0,50.0,200.0,20.0,2,1.872072,68.0,False,52.534996,13.417579,62,True,False,False,True,True,True,True,True,False,False,1,2,1,0,0
3,3,2,1.0,1,26.0,30.0,250.0,18.0,1,4.456289,26.0,False,52.498855,13.349065,5,False,False,False,True,False,True,True,True,False,False,1,5,0,0,1
4,4,2,1.0,1,42.0,0.0,0.0,24.0,1,2.663910,20.0,True,52.543157,13.415091,2,False,False,False,True,False,False,True,True,False,False,1,2,0,2,0


In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [4]:
df.dtypes

accommodates                  int64
bathrooms                   float64
bedrooms                      int64
price                       float64
cleaning_fee                float64
security_deposit            float64
extra_people                float64
guests_included               int64
distance                    float64
size                        float64
host_is_superhost              bool
latitude                    float64
longitude                   float64
minimum_nights                int64
instant_bookable               bool
is_business_travel_ready       bool
tv                             bool
wifi                           bool
kitchen                        bool
washer                         bool
dryer                          bool
heating                        bool
free_parking                   bool
smoking_allowed                bool
property_type                 int64
neighbourhood                 int64
room_type                     int64
cancellation_policy         

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
from sklearn.model_selection import train_test_split

X = df.drop(["price"],axis=1)
y = df["price"]
sc.fit(X)

X_scaled = sc.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.10,random_state=42)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers, metrics
from tensorflow.keras.callbacks import EarlyStopping

t_model = Sequential()
t_model.add(Dense(80, activation="relu", kernel_initializer='normal', input_shape=(28,)))
t_model.add(Dropout(0.2))
t_model.add(Dense(120, activation="relu", kernel_initializer='normal', 
    kernel_regularizer=regularizers.l1(0.01), bias_regularizer=regularizers.l1(0.01)))
t_model.add(Dropout(0.2))
t_model.add(Dense(20, activation="relu", kernel_initializer='normal', 
    kernel_regularizer=regularizers.l1_l2(0.01), bias_regularizer=regularizers.l1_l2(0.01)))
t_model.add(Dropout(0.1))
t_model.add(Dense(10, activation="relu", kernel_initializer='normal'))
t_model.add(Dropout(0.1))
t_model.add(Dense(1))
t_model.compile(
    loss='mean_absolute_error',
    optimizer='nadam',
    metrics=[metrics.mae])

t_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 80)                2320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 120)               9720      
_________________________________________________________________
dropout_5 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 20)                2420      
_________________________________________________________________
dropout_6 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)               

In [14]:
t_model.fit(X_train, y_train,
    batch_size=125,
    epochs=500,
    shuffle=True,
    verbose=1,
    validation_data=(X_test, y_test))

Epoch 1/500
161/161 [==============================] - 1s 5ms/step - loss: 40.5827 - mean_absolute_error: 38.4941 - val_loss: 25.2559 - val_mean_absolute_error: 23.6026
Epoch 2/500
161/161 [==============================] - 1s 4ms/step - loss: 26.1165 - mean_absolute_error: 24.5079 - val_loss: 22.8106 - val_mean_absolute_error: 21.1710
Epoch 3/500
161/161 [==============================] - 1s 4ms/step - loss: 24.0887 - mean_absolute_error: 22.4308 - val_loss: 21.2892 - val_mean_absolute_error: 19.6237
Epoch 4/500
161/161 [==============================] - 1s 4ms/step - loss: 23.3120 - mean_absolute_error: 21.6817 - val_loss: 20.9558 - val_mean_absolute_error: 19.4041
Epoch 5/500
161/161 [==============================] - 1s 4ms/step - loss: 22.8678 - mean_absolute_error: 21.3275 - val_loss: 20.9219 - val_mean_absolute_error: 19.4544
Epoch 6/500
161/161 [==============================] - 1s 4ms/step - loss: 22.6041 - mean_absolute_error: 21.1227 - val_loss: 20.5884 - val_mean_absolute_e

In [ ]:
# !pip install tensorflow

In [22]:
scores = t_model.evaluate(X_test,y_test)
# print(f"{t_model.metrics_names[1]}: {scores[1]*100}")

index = 90

X_pred = sc.transform([df.drop('price', axis=1).iloc[index]])
y_real = df.iloc[index]['price']
y_pred = t_model.predict([X_pred])

print(f'Prediction: ${y_pred[0][0]} | Real: ${y_real}')


70/70 [==============================] - 0s 1ms/step - loss: 17.3441 - mean_absolute_error: 16.9494
Prediction: $58.469940185546875 | Real: $55.0


In [16]:
df['price'].mean()

57.588593155893534

In [17]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((20119, 28), (20119,), (2236, 28), (2236,))

In [24]:
def save_model():
    # serialize model to JSON
    model_json = t_model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    t_model.save_weights("model.h5")
    print("Saved model to disk")

save_model()

Saved model to disk


In [ ]:
from pickle import dump

# save the model
# dump(t_model, open('model.pkl', 'wb'))
# save the scaler
dump(sc, open('scaler.pkl', 'wb'))

In [ ]:
from tensorflow.keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")

In [29]:
model.compile(
    loss='mean_absolute_error',
    optimizer='nadam',
    metrics=[metrics.mae])
scores = model.evaluate(X_test,y_test)
# print(f"{t_model.metrics_names[1]}: {scores[1]*100}")

index = 90

X_pred = sc.transform([df.drop('price', axis=1).iloc[index]])
y_real = df.iloc[index]['price']
y_pred = model.predict([X_pred])

print(f'Prediction: ${y_pred[0][0]} | Real: ${y_real}')


70/70 [==============================] - 0s 1ms/step - loss: 17.3441 - mean_absolute_error: 16.9494
Prediction: $58.469940185546875 | Real: $55.0


In [ ]:
from pickle import load

# load the model
# model = load(open('model.pkl', 'rb'))
# load the scaler
scaler = load(open('scaler.pkl', 'rb'))